<a href="https://colab.research.google.com/github/JeonJunWooo/mini_project/blob/main/%ED%94%8C%EC%A0%9D3%EC%B0%A8_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# ROOT_PATH 확인 
import os

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/AIVLE_3rd_Team"
else :
  ROOT_PATH = os.path.abspath('..')

# 비디오 파일이 저장되어 있는 폴더
VIDEO_PATH = ROOT_PATH + "/video/train/train_video"

# 모델 학습을 위한 데이터 
TRAIN_PATH = ROOT_PATH + "/train/image"
# 모델 예측을 위한 test 데이터
TEST_PATH = ROOT_PATH + "/test"
# MODEL 저장 경로
MODEL_PATH = ROOT_PATH + "/model"

## 1. ImageDataGenerator 생성하기

In [4]:
from numpy.random import shuffle
# 실습해보세요.


batch_size = 16
img_height = 480
img_width = 854


train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # 이미지 데이터 정규화
    validation_split=0.2, # train, validation 데이터 분할 (8:2)
    fill_mode='nearest',


)

# train_genrator 생성
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    batch_size = 16,
    target_size=(480, 854),
    class_mode = 'categorical',
    subset = 'training',
    color_mode= 'grayscale'
)

# validation_generator 생성
validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH, 
    batch_size= 16,
    target_size=(img_height, img_width),
    class_mode = 'categorical',
    subset = 'validation',
    color_mode= 'grayscale'
)

Found 503 images belonging to 4 classes.
Found 123 images belonging to 4 classes.


## 모델 구성하기

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.losses import categorical_crossentropy, sparse_categorical_crossentropy

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.layers import BatchNormalization, Dropout

import random as rd
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers

In [6]:
img_height, img_width

(480, 854)

In [8]:
## 세션클리어
clear_session()

## 모델 선언

model = keras.models.Sequential()
# 인풋레이어
model.add(keras.layers.Input(shape=(img_height,img_width,1)))

model.add(keras.layers.Conv2D(filters=32,
                              kernel_size = (3,3),
                              padding = 'same',
                              activation = 'relu',
                              ))
BatchNormalization()
model.add(keras.layers.MaxPool2D(pool_size=(2,2),
                                 strides = (3,3)))
Dropout(0.25)
model.add(keras.layers.Conv2D(filters=32,
                              kernel_size = (3,3),
                              padding = 'same',
                              activation = 'relu',
                              ))
model.add(keras.layers.MaxPool2D(pool_size=(2,2),
                                 strides = (3,3)))
Dropout(0.25)
model.add( layers.Flatten() )
model.add( layers.Dense(16, activation='relu') )
BatchNormalization()
Dropout(0.35)

# 아웃풋레이어
model.add( layers.Dense(4, activation='softmax') )



##요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 480, 854, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 160, 285, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 160, 285, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 95, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 161120)            0         
                                                                 
 dense (Dense)               (None, 16)                2

In [9]:
# 모델 컴파일 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy']
              )

# early_stopping 

checkpoint = ModelCheckpoint(
    filepath=MODEL_PATH,
    monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
    verbose=1,            # 로그를 출력합니다
    save_best_only=True,  # 가장 best 값만 저장합니다
    mode='auto'           # auto는 알아서 best를 찾습니다. min/max
)


# early_stopping
early_stopping = EarlyStopping(
    monitor= 'val_loss',
    min_delta = 0,
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)


In [10]:
history = model.fit_generator(
	train_generator,
    steps_per_epoch=15, #한 번의 epoch를 돌 때, 데이터를 몇 번 볼 것인가
    epochs=100,
    verbose=1,
    callbacks=[early_stopping,checkpoint],
    validation_data = validation_generator,
        )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/100
15/15 [==============================] - ETA: 0s - loss: 1.0304 - accuracy: 0.6710
Epoch 1: val_loss improved from inf to 0.47005, saving model to /content/drive/MyDrive/AIVLE_3rd_Team/model
15/15 [==============================] - 84s 5s/step - loss: 1.0304 - accuracy: 0.6710 - val_loss: 0.4700 - val_accuracy: 0.7642
Epoch 2/100
15/15 [==============================] - ETA: 0s - loss: 0.2006 - accuracy: 0.9458
Epoch 2: val_loss improved from 0.47005 to 0.06377, saving model to /content/drive/MyDrive/AIVLE_3rd_Team/model
15/15 [==============================] - 25s 2s/step - loss: 0.2006 - accuracy: 0.9458 - val_loss: 0.0638 - val_accuracy: 0.9919
Epoch 3/100
15/15 [==============================] - ETA: 0s - loss: 0.0069 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.06377
15/15 [==============================] - 12s 762ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0702 - val_accuracy: 0.9919
Epoch 4/100
15/15 [==============================] - ETA: 

In [11]:
# 가중치 저장
model.save_weights(MODEL_PATH + '/my_weight.h5')

#  weight만 불러오기
model.load_weights(MODEL_PATH + '/my_weight.h5')


#  모델 통째로 저장
model.save(MODEL_PATH + '/팀_미니프로젝트3차_a206205_전준우.h5')

